# TensorFlow playground
... Just see how it works and where it is useful...

## MNIST with Softmax Regression

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.__version__

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
mnist.train.images.shape

In [2]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [3]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [4]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [5]:
init = tf.initialize_all_variables()

In [6]:
sess = tf.Session()
sess.run(init)

In [7]:
xs = np.load('images.npy')
ys = np.load('labels.npy')

In [9]:
for i in range(10):
    sess.run(train_step, feed_dict={x: xs, y_: ys})

In [ ]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# sess.run(accuracy, feed_dict={x: xs, y_: ys})
sess.run(accuracy, feed_dict={x: xs, y_: ys})

In [ ]:
sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})

## MNIST with Multilayer Convolutional Network


I don't understand how positive initial bias work as described as "Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons." https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_prob = tf.nn.dropout(h_fc1, keep_prob)

In [ ]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_prob, W_fc2) + b_fc2)

In [ ]:
cross_entropy = tf.reduce_mean(- tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())

In [ ]:
xs = np.load('images.npy')
ys = np.load('labels.npy')

In [ ]:
train_step.run(feed_dict={x: xs, y_: ys, keep_prob: 0.8})

In [ ]:
print("test accuracy %g" % accuracy.eval(feed_dict={x: xs, y_: ys, keep_prob: 1.0}))

In [ ]:
for i in range(2000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
#         saver.save(sess, '/Users/yamato/OneDrive/MyCode/MNIST', global_step=i)
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, train accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

In [ ]:
print("test accuracy %g" % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
saver = tf.train.Saver()